In [ ]:
# Setup the library
import pyrogue as pr
import os, sys
import matplotlib.pyplot as plt
import time
import numpy as np
import math

top_level=f'{os.getcwd()}/../../../'
pr.addLibraryPath( f'{top_level}/scripts' )
import setupLibPaths
import epix_hr_single_10k
import ePixViewer.Cameras as cameras

%run ../supportFunctions.ipynb

In [ ]:
# Create and start the root class
root = epix_hr_single_10k.RootLCLSIITiming(
    top_level   = top_level,
    sim         = False,
    dev         = '/dev/datadev_0',
    asicVersion = 4
)
root.start()

In [ ]:
# Create useful pointers
AxiVersion = root.Core.AxiVersion
print ( '###################################################')
print ( '#             Firmware Version                    #')
print ( '###################################################')
AxiVersion.printStatus()
print ( '###################################################')

# Baseline TEST

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(10)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)


In [ ]:
#use system with batcher and missing ASICs
root.EpixHR.SspLowSpeedDecoderReg.enable.set(True)
print ("Locked lanes: %x" % root.EpixHR.SspLowSpeedDecoderReg.Locked.get())
#Only ASIC 1 is available so disable the 6 lower lanes
root.EpixHR.PacketRegisters0.DisableLane.set(0x3FF)
root.EpixHR.PacketRegisters1.DisableLane.set(0xFC0)
#using internal triggers, so bypassing timing channel at the batcher 
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)


In [ ]:
print(ASIC.Preamp.get())
print(ASIC.Vld1_b.get())
print(ASIC.Pixel_CB.get())
print(ASIC.Filter_DAC.get())
print(ASIC.VRef_DAC.get())

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())
DHg = True
RefGenB = 2
RefGenC = 3
S2D_1_b = 0
shvc_DAC = 23
S2dDacBias = 3
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [7,6,5,4,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#scatter plot
plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,10,10]
plt.plot(dataset, 'rx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,140,75]
plt.plot(dataset, 'bx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


In [ ]:
noise_print_roi(x=65,y=134,width=40,height=10)

In [ ]:
noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
plt.figure(1,figsize=(6,4),dpi=150)
plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffc
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
# baseline test with external DAC
print(currentCam.bitMask)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    
    ASIC.mTest.set(True)

    #sets HSDAC device
    root.EpixHR.HSDac.enable.set(True)
    #reset status
    root.EpixHR.HSDac.DacChannel.set(1) # 0 means NONE, 1 means SE, 2 Diff, 3 both
    root.EpixHR.HSDac.WFEnabled.set(False)
    root.EpixHR.HSDac.waveformSource.set(0)
    root.EpixHR.HSDac.externalUpdateEn.set(False)
    root.EpixHR.HSDac.run.set(False)

    root.EpixHR.HSDac.DacValue.set(32000)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(20000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


# Baseline Matrix in RESET TEST

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(10)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)


In [ ]:
#use system with batcher and missing ASICs
root.EpixHR.SspLowSpeedDecoderReg.enable.set(True)
print ("Locked lanes: %x" % root.EpixHR.SspLowSpeedDecoderReg.Locked.get())
#Only ASIC 1 is available so disable the 6 lower lanes
root.EpixHR.PacketRegisters0.DisableLane.set(0x3FF)
root.EpixHR.PacketRegisters1.DisableLane.set(0xFC0)
#using internal triggers, so bypassing timing channel at the batcher 
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)


In [ ]:
print(ASIC.Preamp.get())
print(ASIC.Vld1_b.get())
print(ASIC.Pixel_CB.get())
print(ASIC.Filter_DAC.get())
print(ASIC.VRef_DAC.get())

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())
DHg = True
RefGenB = 2
RefGenC = 3
S2D_1_b = 0
shvc_DAC = 23
S2dDacBias = 3
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [7,6,5,4,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(2)

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#scatter plot
plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,10,10]
plt.plot(dataset, 'rx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,140,75]
plt.plot(dataset, 'bx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


In [ ]:
noise_print_roi(x=65,y=134,width=40,height=10)

In [ ]:
noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
plt.figure(1,figsize=(6,4),dpi=150)
plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffc
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
# baseline test with external DAC
print(currentCam.bitMask)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    
    ASIC.mTest.set(True)

    #sets HSDAC device
    root.EpixHR.HSDac.enable.set(True)
    #reset status
    root.EpixHR.HSDac.DacChannel.set(1) # 0 means NONE, 1 means SE, 2 Diff, 3 both
    root.EpixHR.HSDac.WFEnabled.set(False)
    root.EpixHR.HSDac.waveformSource.set(0)
    root.EpixHR.HSDac.externalUpdateEn.set(False)
    root.EpixHR.HSDac.run.set(False)

    root.EpixHR.HSDac.DacValue.set(32000)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(20000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


# Pipo Delay test

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
ASIC.pipoclk_delay.get()

In [ ]:
ASIC.mTest.set(True)

#sets HSDAC device
root.EpixHR.HSDac.enable.set(True)
#reset status
root.EpixHR.HSDac.DacChannel.set(1) # 0 means NONE, 1 means SE, 2 Diff, 3 both
root.EpixHR.HSDac.WFEnabled.set(False)
root.EpixHR.HSDac.waveformSource.set(0)
root.EpixHR.HSDac.externalUpdateEn.set(False)
root.EpixHR.HSDac.run.set(False)

root.EpixHR.HSDac.DacValue.set(32000)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 4
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53
pipoDelayList = np.arange(15)

for pipoDelay in pipoDelayList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    print(pipoDelay)
    ASIC.pipoclk_delay.set(int(pipoDelay))

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_pipoDelay_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC, pipoDelay)) 
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(numFrames=2000,initialDelay=2.0)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = glitchFinder_print(x=0,y=0,width=144,height=192)

# Baseline TEST all

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(10)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)


In [ ]:
#use system with batcher and missing ASICs
root.EpixHR.SspLowSpeedDecoderReg.enable.set(True)
print ("Locked lanes: %x" % root.EpixHR.SspLowSpeedDecoderReg.Locked.get())
#Only ASIC 1 is available so disable the 6 lower lanes
root.EpixHR.PacketRegisters0.DisableLane.set(0x3FF)
root.EpixHR.PacketRegisters1.DisableLane.set(0xFC0)



In [ ]:
print(ASIC.Preamp.get())
print(ASIC.Vld1_b.get())
print(ASIC.Pixel_CB.get())
print(ASIC.Filter_DAC.get())
print(ASIC.VRef_DAC.get())

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())
DHg = True
RefGenB = 2
RefGenC = 3
S2D_1_b = 0
shvc_DAC = 23
S2dDacBias = 3
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

In [ ]:
def complete_test(filename):
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0,imgDesc=imgDesc)
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()

    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()

    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=65,y=134,width=40,height=10, imgDesc=imgDesc)    
    
    noise_print_roi(x=10,y=10,width=50,height=50, imgDesc=imgDesc)
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)
    
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0, imgDesc=imgDesc)
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50,imgDesc=imgDesc)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [7,6,5,4,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_bList = [3,2,1,0] #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Vld1_b in Vld1_bList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_b = 1 #1
Pixel_CBList= [7,6,5,4,3,2,1,0] #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Pixel_CB in Pixel_CBList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DACList= [53,48,43,38,33,28,23,18] #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Filter_DAC in Filter_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DACList = [53,48,43,38,33,28,23,18] #22

ASIC.ClearMatrix(0)

for VRef_DAC in VRef_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




# 2

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 7
Vld1_bList = [0, 1, 2, 3]
Pixel_CB= 7
Filter_DAC= 17
VRef_DAC = 58

for Vld1_b in Vld1_bList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 7
Vld1_b = 0
Pixel_CBList= [7,6,5,4,3,2,1,0]
Filter_DAC= 17
VRef_DAC = 58

for Pixel_CB in Pixel_CBList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 7
Vld1_b = 0
Pixel_CB= 7
Filter_DACList=[7, 13, 15, 17, 19, 21, 27, 37, 47, 57]
VRef_DAC = 58

for Filter_DAC in Filter_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
Preamp = 7
Vld1_b = 0
Pixel_CB= 7
Filter_DAC= 17
VRef_DACList =[3, 13, 23, 33, 43, 48, 53, 58, 63]

for VRef_DAC in VRef_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()